<a href="https://colab.research.google.com/github/marina554/Consolidated-Accounting/blob/main/Handles_both_B_S_and_P_L.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# =========================================
# Simple Consolidated Accounting Model (B/S + P/L)
# =========================================
import pandas as pd

# ---------------------------
# 1. Individual Financial Data
# ---------------------------

# Parent Company
parent_bs = {
    "Cash": 1000,
    "Accounts_Receivable": 500,
    "Inventory": 300,
    "Investment_in_Sub": 300,  # Investment account (simplified here)
    "Accounts_Payable": 400,
    "Intercompany_Receivable": 200,  # For internal transactions
    "Capital_Stock": 1000,
    "Retained_Earnings": 700
}

parent_pl = {
    "Revenue": 2000,
    "COGS": 800,
    "SGA": 300,
    "Intercompany_Sales": 200,  # Internal sales to subsidiary
}

# Subsidiary
child_bs = {
    "Cash": 300,
    "Accounts_Receivable": 100,
    "Inventory": 100,
    "Intercompany_Payable": 200,  # Internal transactions with parent
    "Accounts_Payable": 150,
    "Capital_Stock": 200,
    "Retained_Earnings": 150
}

child_pl = {
    "Revenue": 500,
    "COGS": 200,
    "SGA": 100,
    "Intercompany_Purchases": 200  # Internal purchases from parent
}

# Ownership ratio (percentage of subsidiary owned by parent)
ownership = 0.6  # 60%

# ---------------------------
# 2. Internal Transaction Elimination Function (B/S & P/L)
# ---------------------------
def eliminate_internal_transactions(parent_pl, child_pl, parent_bs, child_bs):
    """
    Eliminate internal sales/purchases and receivables/payables between parent and subsidiary.
    """
    # Assume internal transaction amounts are equal
    internal_amount = min(parent_pl["Intercompany_Sales"], child_pl["Intercompany_Purchases"])

    # Eliminate sales and purchases
    parent_pl["Intercompany_Sales"] -= internal_amount
    child_pl["Intercompany_Purchases"] -= internal_amount

    # Eliminate intercompany receivables/payables
    internal_bs_amount = min(parent_bs["Intercompany_Receivable"], child_bs["Intercompany_Payable"])
    parent_bs["Intercompany_Receivable"] -= internal_bs_amount
    child_bs["Intercompany_Payable"] -= internal_bs_amount

    print(f"Internal transaction elimination (Revenue/Purchases): {internal_amount}")
    print(f"Internal transaction elimination (Receivables/Payables): {internal_bs_amount}")

# Execute elimination
eliminate_internal_transactions(parent_pl, child_pl, parent_bs, child_bs)

# ---------------------------
# 3. Create Consolidated B/S
# ---------------------------
def create_consolidated_bs(parent, child, ownership):
    consolidated = {}
    for key in set(parent.keys()) | set(child.keys()):
        consolidated[key] = parent.get(key, 0) + child.get(key, 0)

    # Non-controlling interest (subsidiary net assets × (1 - ownership))
    child_net_assets = child["Capital_Stock"] + child["Retained_Earnings"]
    nci_bs = child_net_assets * (1 - ownership)
    consolidated["Noncontrolling_Interest"] = nci_bs
    return consolidated

# ---------------------------
# 4. Create Consolidated P/L
# ---------------------------
def create_consolidated_pl(parent_pl, child_pl, ownership):
    consolidated = {}
    for key in set(parent_pl.keys()) | set(child_pl.keys()):
        consolidated[key] = parent_pl.get(key, 0) + child_pl.get(key, 0)

    # Internal transactions already eliminated
    consolidated.pop("Intercompany_Sales", None)
    consolidated.pop("Intercompany_Purchases", None)

    # Operating income (simplified)
    consolidated["Operating_Income"] = (
        consolidated["Revenue"]
        - consolidated["COGS"]
        - consolidated["SGA"]
    )

    # Subsidiary net income for non-controlling interest
    child_net_income = child_pl["Revenue"] - child_pl["COGS"] - child_pl["SGA"]
    nci_pl = child_net_income * (1 - ownership)
    consolidated["Noncontrolling_Interest_Net_Income"] = nci_pl
    consolidated["Net_Income_Attributable_to_Parent"] = consolidated["Operating_Income"] - nci_pl

    return consolidated

# ---------------------------
# 5. Execute and Display Results
# ---------------------------
consolidated_bs = create_consolidated_bs(parent_bs, child_bs, ownership)
consolidated_pl = create_consolidated_pl(parent_pl, child_pl, ownership)

df_bs = pd.DataFrame([consolidated_bs], index=["Consolidated B/S"])
df_pl = pd.DataFrame([consolidated_pl], index=["Consolidated P/L"])

print("\n==== Consolidated Balance Sheet (B/S) ====")
print(df_bs.T)

print("\n==== Consolidated Income Statement (P/L) ====")
print(df_pl.T)


Internal transaction elimination (Revenue/Purchases): 200
Internal transaction elimination (Receivables/Payables): 200

==== Consolidated Balance Sheet (B/S) ====
                         Consolidated B/S
Intercompany_Receivable               0.0
Accounts_Payable                    550.0
Investment_in_Sub                   300.0
Accounts_Receivable                 600.0
Cash                               1300.0
Intercompany_Payable                  0.0
Capital_Stock                      1200.0
Inventory                           400.0
Retained_Earnings                   850.0
Noncontrolling_Interest             140.0

==== Consolidated Income Statement (P/L) ====
                                    Consolidated P/L
COGS                                          1000.0
SGA                                            400.0
Revenue                                       2500.0
Operating_Income                              1100.0
Noncontrolling_Interest_Net_Income              80.0
Net_Income

In [1]:
# =========================================
# 連結会計シンプルモデル（B/S＋P/L対応版）
# =========================================
import pandas as pd

# ---------------------------
# 1. 各社の単体財務データ
# ---------------------------

# 親会社
parent_bs = {
    "現金": 1000,
    "売掛金": 500,
    "棚卸資産": 300,
    "子会社株式": 300,  # 投資勘定（今回は簡略化）
    "買掛金": 400,
    "親子間売掛金": 200,  # 内部取引用
    "資本金": 1000,
    "利益剰余金": 700
}

parent_pl = {
    "売上高": 2000,
    "売上原価": 800,
    "販売費及び一般管理費": 300,
    "親子間売上高": 200,  # 子会社向けの内部売上
}

# 子会社
child_bs = {
    "現金": 300,
    "売掛金": 100,
    "棚卸資産": 100,
    "親子間買掛金": 200,  # 親会社への内部取引
    "買掛金": 150,
    "資本金": 200,
    "利益剰余金": 150
}

child_pl = {
    "売上高": 500,
    "売上原価": 200,
    "販売費及び一般管理費": 100,
    "親子間仕入": 200  # 親会社からの内部仕入
}

# 持分比率（親会社が子会社株式を保有する割合）
ownership = 0.6  # 60%

# ---------------------------
# 2. 内部取引消去関数（B/S & P/L）
# ---------------------------
def eliminate_internal_transactions(parent_pl, child_pl, parent_bs, child_bs):
    """
    親子間の内部売上・仕入、および債権債務を消去
    """
    # 内部取引金額を同額と仮定
    internal_amount = min(parent_pl["親子間売上高"], child_pl["親子間仕入"])

    # 売上・仕入の消去
    parent_pl["親子間売上高"] -= internal_amount
    child_pl["親子間仕入"] -= internal_amount

    # 債権債務の消去
    internal_bs_amount = min(parent_bs["親子間売掛金"], child_bs["親子間買掛金"])
    parent_bs["親子間売掛金"] -= internal_bs_amount
    child_bs["親子間買掛金"] -= internal_bs_amount

    print(f"内部取引消去（売上・仕入）: {internal_amount}")
    print(f"内部取引消去（債権・債務）: {internal_bs_amount}")

# 実行
eliminate_internal_transactions(parent_pl, child_pl, parent_bs, child_bs)

# ---------------------------
# 3. 連結B/S作成
# ---------------------------
def create_consolidated_bs(parent, child, ownership):
    consolidated = {}
    for key in set(parent.keys()) | set(child.keys()):
        consolidated[key] = parent.get(key, 0) + child.get(key, 0)

    # 非支配株主持分（子会社の純資産×持分非保有率）
    child_net_assets = child["資本金"] + child["利益剰余金"]
    nci_bs = child_net_assets * (1 - ownership)
    consolidated["非支配株主持分"] = nci_bs
    return consolidated

# ---------------------------
# 4. 連結P/L作成
# ---------------------------
def create_consolidated_pl(parent_pl, child_pl, ownership):
    consolidated = {}
    for key in set(parent_pl.keys()) | set(child_pl.keys()):
        consolidated[key] = parent_pl.get(key, 0) + child_pl.get(key, 0)

    # 内部取引項目は既に消去済なので無視
    consolidated.pop("親子間売上高", None)
    consolidated.pop("親子間仕入", None)

    # 当期純利益（超シンプル計算）
    consolidated["営業利益"] = (
        consolidated["売上高"]
        - consolidated["売上原価"]
        - consolidated["販売費及び一般管理費"]
    )

    # 子会社当期純利益（非支配株主持分のため）
    child_net_income = child_pl["売上高"] - child_pl["売上原価"] - child_pl["販売費及び一般管理費"]
    nci_pl = child_net_income * (1 - ownership)
    consolidated["非支配株主持分当期純利益"] = nci_pl
    consolidated["親会社株主に帰属する当期純利益"] = consolidated["営業利益"] - nci_pl

    return consolidated

# ---------------------------
# 5. 実行・結果表示
# ---------------------------
consolidated_bs = create_consolidated_bs(parent_bs, child_bs, ownership)
consolidated_pl = create_consolidated_pl(parent_pl, child_pl, ownership)

df_bs = pd.DataFrame([consolidated_bs], index=["連結B/S"])
df_pl = pd.DataFrame([consolidated_pl], index=["連結P/L"])

print("\n==== 連結貸借対照表（B/S） ====")
print(df_bs.T)

print("\n==== 連結損益計算書（P/L） ====")
print(df_pl.T)


内部取引消去（売上・仕入）: 200
内部取引消去（債権・債務）: 200

==== 連結貸借対照表（B/S） ====
          連結B/S
売掛金       600.0
資本金      1200.0
利益剰余金     850.0
親子間買掛金      0.0
棚卸資産      400.0
買掛金       550.0
子会社株式     300.0
現金       1300.0
親子間売掛金      0.0
非支配株主持分   140.0

==== 連結損益計算書（P/L） ====
                  連結P/L
売上原価             1000.0
売上高              2500.0
販売費及び一般管理費        400.0
営業利益             1100.0
非支配株主持分当期純利益       80.0
親会社株主に帰属する当期純利益  1020.0
